In [1]:
# Importing packages and initializing workspace
import gc

import numpy as np
import pandas as pd
import re
from textblob import TextBlob


price = {}
btc_train = pd.read_csv('btc_tweets_mar_2021.csv')
# btc_test = pd.read_csv('btc_tweets_apr_2021.csv')
price['BTC'] = pd.read_csv('FinancialData/Bitstamp_BTCUSD_minute.csv')

eth_train = pd.read_csv('eth_tweets_mar_2021.csv')
# eth_test = pd.read_csv('eth_tweets_apr_2021.csv')
price['ETH'] = pd.read_csv('FinancialData/Bitstamp_ETHUSD_minute.csv')

ltc_train = pd.read_csv('ltc_tweets_mar_2021.csv')
# ltc_test = pd.read_csv('ltc_tweets_apr_2021.csv')
price['LTC'] = pd.read_csv('FinancialData/Bitstamp_LTCUSD_minute.csv')

xrp_train = pd.read_csv('xrp_tweets_mar_2021.csv')
# xrp_test = pd.read_csv('xrp_tweets_apr_2021.csv')
price['XRP'] = pd.read_csv('FinancialData/Bitstamp_XRPUSD_minute.csv')

In [63]:
btc_train.head()

,Datetime,Tweet Id,Text,Username,Language,Followers
0,2021-03-30 23:59:59+00:00,1377047987919187969,"Eventually, I think it will be inevitable I bu...",Memeptah,en,999
1,2021-03-30 23:59:54+00:00,1377047969606828033,#Investing 79.4% in this optimal #crypto portf...,CryptoWatchBot,en,3065
2,2021-03-30 23:59:48+00:00,1377047941672808450,Stock To Flow - Measuring The Bitcoin Peak Usi...,mlleroulleau,en,5207
3,2021-03-30 23:59:35+00:00,1377047890053369858,I like rubies/sapphires/emeralds/diamonds most...,takeck89,en,382
4,2021-03-30 23:59:34+00:00,1377047884303114243,My favorite food and crypto are: https://t.co/...,koin_pencari,en,290


In [2]:
# clean tweets
data_files = [('btc_march.csv', btc_train), ('eth_march.csv', eth_train), ('ltc_march.csv', ltc_train), ('xrp_march.csv', xrp_train)]
# data_files = [('btc_april.csv', btc_test), ('eth_april.csv', eth_test), ('ltc_april.csv', ltc_test), ('xrp_april.csv', xrp_test),
#                 ('btc_march.csv', btc_train), ('eth_march.csv', eth_train), ('ltc_march.csv', ltc_train), ('xrp_march.csv', xrp_train)]
for df_data in data_files:
    print('Cleaning for: ' + df_data[0])
    df_data[1]["Text"] = [text.lower() for text in df_data[1]["Text"]]
    df_data[1]["Text"] = [re.sub("@[A-Za-z0-9_]+","", text) for text in df_data[1]["Text"]]
    df_data[1]["Text"] = [re.sub(r"#","", text) for text in df_data[1]["Text"]]
    df_data[1]["Text"] = [re.sub(r"http\S+", "", text) for text in df_data[1]["Text"]]
    df_data[1]["Text"] = [re.sub(r"www.\S+", "", text) for text in df_data[1]["Text"]]
    df_data[1]["Text"] = [re.sub(r"\n", "", text) for text in df_data[1]["Text"]]

    # get sentiment
    print('Sentiment for: ' + df_data[0])
    df_data[1]["TextBlob"] = [TextBlob(text) for text in df_data[1]["Text"]]
    df_data[1]["Sentiment"] = [text.sentiment for text in df_data[1]["TextBlob"]]
    df_data[1]['Polarity'] = [polarity[0] for polarity in df_data[1]['Sentiment']]
    df_data[1]['Subjectivity'] = [polarity[1] for polarity in df_data[1]['Sentiment']]

    # Clean up df
    df_data[1].drop(['Tweet Id', 'Text', 'Username', 'Language', 'Followers', 'TextBlob', 'Sentiment'], axis=1, inplace=True)
    df_data[1].to_csv(df_data[0])
    print('Saved: ' + df_data[0])

Cleaning for: btc_march.csv
Sentiment for: btc_march.csv
Saved: btc_march.csv
Cleaning for: eth_march.csv
Sentiment for: eth_march.csv
Saved: eth_march.csv
Cleaning for: ltc_march.csv
Sentiment for: ltc_march.csv
Saved: ltc_march.csv
Cleaning for: xrp_march.csv
Sentiment for: xrp_march.csv
Saved: xrp_march.csv


In [3]:
# Now add the price data back onto the sentiment
for curr in price:
    price[curr].drop(['unix', 'symbol', 'open', 'high', 'low', 'Volume ' + curr], axis=1, inplace=True)
    price[curr].rename(columns={'date': 'Timestamp', 'close': 'Price'}, inplace=True)
    price[curr]['Timestamp'] = pd.to_datetime(price[curr]['Timestamp'], infer_datetime_format=True)
    price[curr].set_index('Timestamp', inplace=True)

In [12]:
# Pool the sentiment data into various window lengths 1min, 10min, 1hrs, 1day
btc_train = pd.read_csv('btc_april.csv')
eth_train = pd.read_csv('eth_april.csv')
ltc_train = pd.read_csv('ltc_april.csv')
xrp_train = pd.read_csv('xrp_april.csv')

train_data = {}
test_data = {}

for (code, df_data) in [('BTC', btc_train), ('ETH', eth_train), ('LTC', ltc_train), ('XRP', xrp_train)]:
    df_data['Timestamp'] = pd.to_datetime(df_data['Datetime'], infer_datetime_format=True).dt.tz_localize(None)
    df_data.set_index('Timestamp', inplace=True)
    for interval in ['1min', '10min', '60min', '1d']:
        key = code + '_' + interval
        train_data[key] = df_data.groupby(pd.Grouper(freq=interval)).mean()
        train_data[key] = train_data[key].merge(price[code], how='left', on='Timestamp')
        # train_data[key].drop('Unnamed: 0', axis=1, inplace=True)
        train_data[key].columns = [code + ' ' + str(col) if col != 'Timestamp' else str(col) for col in train_data[key].columns]

del btc_train
del eth_train
del ltc_train
del xrp_train
gc.collect()

# btc_test = pd.read_csv('btc_april.csv')
# eth_test = pd.read_csv('eth_april.csv')
# ltc_test = pd.read_csv('ltc_april.csv')
# xrp_test = pd.read_csv('xrp_april.csv')

# for (code, df_data) in [('BTC', btc_test), ('ETH', eth_test), ('LTC', ltc_test), ('XRP', xrp_test)]:
#     df_data['Timestamp'] = pd.to_datetime(df_data['Datetime'], infer_datetime_format=True).dt.tz_localize(None)
#     df_data.set_index('Timestamp', inplace=True)
#     for interval in ['1min', '10min', '60min', '1d']:
#         key = code + '_' + interval
#         test_data[key] = df_data.groupby(pd.Grouper(freq=interval)).mean()
#         test_data[key] = test_data[key].merge(price[code], how='left', on='Timestamp')
#         test_data[key].columns = [code + ' ' + str(col) if col != 'Timestamp' else str(col) for col in test_data[key].columns]
#
# del btc_test
# del eth_test
# del ltc_test
# del xrp_test
# gc.collect()

284

In [13]:
train_data['BTC_60min']

,BTC Polarity,BTC Subjectivity,BTC Price,BTC Volume USD
Timestamp,,,,
2021-04-01 00:00:00,0.175066,0.389011,58819.71,147060.23610
2021-04-01 01:00:00,0.178756,0.385661,59300.00,120942.26580
2021-04-01 02:00:00,0.192258,0.395874,59256.60,119496.68180
2021-04-01 03:00:00,0.202326,0.431403,59062.92,50524.25213
2021-04-01 04:00:00,0.201704,0.408369,59130.88,24092.55538
...,...,...,...,...
2021-04-29 19:00:00,0.087239,0.267478,52974.90,199495.79050
2021-04-29 20:00:00,0.083779,0.261432,52818.49,198687.77280
2021-04-29 21:00:00,0.088072,0.266146,52886.81,231798.83180


In [14]:
# Now create a net sentiment measure

desired_pairs = [('BTC_1min', 'ETH_1min'), ('BTC_1min', 'LTC_1min'), ('BTC_1min', 'XRP_1min'), ('ETH_1min', 'LTC_1min'), ('ETH_1min', 'XRP_1min'), ('LTC_1min', 'XRP_1min'),('BTC_10min', 'ETH_10min'), ('BTC_10min', 'LTC_10min'), ('BTC_10min', 'XRP_10min'), ('ETH_10min', 'LTC_10min'), ('ETH_10min', 'XRP_10min'), ('LTC_10min', 'XRP_10min'), ('BTC_60min', 'ETH_60min'), ('BTC_60min', 'LTC_60min'), ('BTC_60min', 'XRP_60min'), ('ETH_60min', 'LTC_60min'), ('ETH_60min', 'XRP_60min'), ('LTC_60min', 'XRP_60min'), ('BTC_1d', 'ETH_1d'), ('BTC_1d', 'LTC_1d'), ('BTC_1d', 'XRP_1d'), ('ETH_1d', 'LTC_1d'), ('ETH_1d', 'XRP_1d'), ('LTC_1d', 'XRP_1d')]

for pair in desired_pairs:
    joint_pair = pair[0] + '_' + pair[1]
    curr_1 = pair[0].split('_')[0]
    curr_2 = pair[1].split('_')[0]

    train_data[joint_pair] = train_data[pair[0]].merge(train_data[pair[1]], on='Timestamp')
    train_data[joint_pair]['Net Polarity'] = train_data[joint_pair][curr_1 + ' Polarity'].sub(train_data[joint_pair][curr_2 + ' Polarity'], fill_value=np.nan)
    train_data[joint_pair]['Net Subjectivity'] = train_data[joint_pair][curr_1 + ' Subjectivity'].sub(train_data[joint_pair][curr_2 + ' Subjectivity'], fill_value=np.nan)
    train_data[joint_pair]['Net Volume'] = train_data[joint_pair][curr_1 + ' Volume USD'].sub(train_data[joint_pair][curr_2 + ' Volume USD'], fill_value=np.nan)
    train_data[joint_pair]['Price Ratio'] = train_data[joint_pair][curr_2 + ' Price'].div(train_data[joint_pair][curr_1 + ' Price'], fill_value=np.nan)
    train_data[joint_pair].to_csv(joint_pair + '_april.csv')
    print('Saved Train: ' + joint_pair)

    # test_data[joint_pair] = test_data[pair[0]].merge(test_data[pair[1]], on='Timestamp')
    # test_data[joint_pair]['Net Polarity'] = test_data[joint_pair][curr_1 + ' Polarity'].sub(test_data[joint_pair][curr_2 + ' Polarity'], fill_value=np.nan)
    # test_data[joint_pair]['Net Subjectivity'] = test_data[joint_pair][curr_1 + ' Subjectivity'].sub(test_data[joint_pair][curr_2 + ' Subjectivity'], fill_value=np.nan)
    # test_data[joint_pair]['Net Volume'] = test_data[joint_pair][curr_1 + ' Volume USD'].sub(test_data[joint_pair][curr_2 + ' Volume USD'], fill_value=np.nan)
    # test_data[joint_pair]['Price Ratio'] = test_data[joint_pair][curr_2 + ' Price'].div(test_data[joint_pair][curr_1 + ' Price'], fill_value=np.nan)
    # test_data[joint_pair].to_csv(joint_pair + '_april.csv')
    # print('Saved Test: ' + joint_pair)


Saved Train: BTC_1min_ETH_1min
Saved Train: BTC_1min_LTC_1min
Saved Train: BTC_1min_XRP_1min
Saved Train: ETH_1min_LTC_1min
Saved Train: ETH_1min_XRP_1min
Saved Train: LTC_1min_XRP_1min
Saved Train: BTC_10min_ETH_10min
Saved Train: BTC_10min_LTC_10min
Saved Train: BTC_10min_XRP_10min
Saved Train: ETH_10min_LTC_10min
Saved Train: ETH_10min_XRP_10min
Saved Train: LTC_10min_XRP_10min
Saved Train: BTC_60min_ETH_60min
Saved Train: BTC_60min_LTC_60min
Saved Train: BTC_60min_XRP_60min
Saved Train: ETH_60min_LTC_60min
Saved Train: ETH_60min_XRP_60min
Saved Train: LTC_60min_XRP_60min
Saved Train: BTC_1d_ETH_1d
Saved Train: BTC_1d_LTC_1d
Saved Train: BTC_1d_XRP_1d
Saved Train: ETH_1d_LTC_1d
Saved Train: ETH_1d_XRP_1d
Saved Train: LTC_1d_XRP_1d


In [15]:
train_data['BTC_10min_ETH_10min']

,BTC Polarity,BTC Subjectivity,BTC Price,BTC Volume USD,ETH Polarity,ETH Subjectivity,ETH Price,ETH Volume USD,Net Polarity,Net Subjectivity,Net Volume,Price Ratio
Timestamp,,,,,,,,,,,,
2021-04-01 00:00:00,0.162737,0.366383,58819.71,147060.23610,0.172430,0.427254,1929.72,100208.998700,-0.009694,-0.060871,46851.237400,0.032807
2021-04-01 00:10:00,0.182590,0.402338,59080.08,15975.59748,0.176927,0.410761,1928.68,1051.576607,0.005662,-0.008424,14924.020873,0.032645
2021-04-01 00:20:00,0.148116,0.363064,58943.56,29414.36485,0.150114,0.442523,1926.58,1009.527920,-0.001998,-0.079459,28404.836930,0.032685
2021-04-01 00:30:00,0.190115,0.408182,59006.44,40951.92151,0.168412,0.448025,1924.97,0.000000,0.021703,-0.039842,40951.921510,0.032623
2021-04-01 00:40:00,0.166590,0.386431,59024.85,40068.05496,0.246038,0.495427,1924.77,34864.531790,-0.079448,-0.108996,5203.523170,0.032609
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-29 23:10:00,0.099791,0.281702,53406.91,139553.93760,0.130814,0.303374,2762.43,10724.563950,-0.031023,-0.021672,128829.373650,0.051724
2021-04-29 23:20:00,0.073965,0.252955,53488.60,91037.61913,0.107149,0.274438,2761.54,1577.251113,-0.033184,-0.021482,89460.368017,0.051629
2021-04-29 23:30:00,0.077567,0.265562,53622.92,72523.29302,0.079611,0.314013,2765.80,12907.782350,-0.002043,-0.048451,59615.510670,0.051579
